###ML1_1: 
https://www.hackerrank.com/challenges/capturing-non-capturing-groups/problem?isFullScreen=true

ML1_2: 
https://www.hackerrank.com/challenges/branch-reset-groups/problem?isFullScreen=true

ML1_3: 
https://www.hackerrank.com/challenges/detect-html-links/problem?isFullScreen=true

ML1_4: Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

In [33]:
#ML1_1
import re

pattern = r'()okokok()'
test_string = 'hi its okokok rly'
result = re.search(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")	

Search successful.


In [1]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 62.9 MB/s 
     |████████████████████████████████| 18.5 MB 593 kB/s 
     |████████████████████████████████| 88 kB 7.8 MB/s 
     |████████████████████████████████| 1.1 MB 66.7 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=2ee78dc810a953bc62e41f7484cc20ea82b2cf179e1451ccb4c2a4a4b345ecdb
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=25c0b6969dc034dfee6718a0aebcec0f4171b6de560fb7fc95afa60024ac3457
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling ge

In [2]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2022-07-20 19:06:08--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-07-20 19:06:08--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8a2bf6db0e005830e21225d55d.dl.dropboxusercontent.com/cd/0/get/BpewCZdtjnPd7g7SNxOMpEAgn8sz3xwvCzviqUm-XuwAHsLSLqWIXClkEu-D0i7Bvb9BnagSMNge5BXtAHtNmC4yuP65B0XUmC5yFU_koHRuNbg2Hdm_3AjbDCMZVK-nJyKVV5t_P8f-7o6j-h3nPov7bbX04-v0OdkITn4SO42STA/file?dl=1# [following]
--2022-07-20 19:06:08--  https://uc8a2bf6db0e005830e21225d55d.dl.dropboxusercontent.com/cd/0/get/BpewCZdtjnPd7g7SNxOMpEAgn8sz3xwvCzviqUm-XuwAHsLSLqWIXClkEu-D0i7Bvb9BnagSMNg

In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [5]:
data_tok = []
for i in range(len(data)): 
  data_tok.append(tokenizer.tokenize(data[i].lower()))


#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"


In [6]:
data_tok[1]

['what',
 'are',
 'some',
 'ways',
 'to',
 'overcome',
 'a',
 'fast',
 'food',
 'addiction',
 '?']

In [7]:
len(data_tok)

537272

###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [8]:
sl = {}
for i in data_tok:
  for j in i:
    if j not in sl:
      sl[j]=1
    else:
      sl[j]+=1

In [9]:
sorted(sl, key=sl.get, reverse=True)[:10]

['?', 'the', 'what', 'is', 'a', 'i', 'to', 'in', 'how', 'of']

In [10]:
import operator
sorted(sl.items(), key=operator.itemgetter(1), reverse=True)[:10]

[('?', 552413),
 ('the', 252068),
 ('what', 214798),
 ('is', 185392),
 ('a', 155726),
 ('i', 149740),
 ('to', 141788),
 ('in', 139787),
 ('how', 135687),
 ('of', 112001)]

In [11]:
import pandas as pd
df = pd.DataFrame.from_dict(sl, orient='index')
df.sort_values(0, ascending = False).head(10)

,0
?,552413
the,252068
what,214798
is,185392
a,155726
i,149740
to,141788
in,139787
how,135687
of,112001


###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [12]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [14]:
lemmatizer.lemmatize("rocks")

'rock'

In [15]:
data_tok_lem=[]
for i in data_tok:
  for j in i:
    data_tok_lem.append(lemmatizer.lemmatize(j))

In [16]:
sl_lem = {}
for i in data_tok_lem:
  if i not in sl_lem:
    sl_lem[i]=1
  else:
    sl_lem[i]+=1

In [17]:
print(len(sl))
print(len(sl_lem))

87819
80303


In [18]:
dict_words={}
for token in data_tok:
  for word in token: 
    word_lemmatizing = lemmatizer.lemmatize(word)
    if word_lemmatizing in dict_words.keys(): 
      dict_words[word_lemmatizing] += 1
    else:
      dict_words[word_lemmatizing] = 1

In [19]:
print(len(dict_words))

80303


###Задание 4: Подсчитайте количество разных слов до и после стемминга

In [20]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [21]:
dict_words={}
for token in data_tok:
  for word in token: 
    word_stemming = ps.stem(word)
    if word_stemming in dict_words.keys(): 
      dict_words[word_stemming] += 1
    else:
      dict_words[word_stemming] = 1

In [24]:
print(len(dict_words))

67026


###Задание 5: Подсчитайте количество разных слов



In [25]:
# Не совсем понял задание. Количество разных слов это изначальный словарь, собранные на основе разных эл-ов data_tok
len(sl)

87819

REGEXP

https://www.programiz.com/python-programming/regex 

https://docs.python.org/3/howto/regex.html

In [22]:
import re

pattern = 'a[ybs]+s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")	

Search successful.


###Задание 6: 
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

###Задание 7: 
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8: 
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9: 
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

Bag Of Words (BOW)

In [23]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two 
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)